### Limpieza general 
normalizar fondo y eliminar archivos no finalizados en .png

In [ ]:
import os
import io
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd


version="1.21.1"
tam=32

item_folder = f"datos/item_{version}_{str(tam)}x{str(tam)}"
block_folder = f"datos/block_{version}_{str(tam)}x{str(tam)}"

BACKGROUND_COLOR = (255, 255, 255)   


In [ ]:
def safe_open_image(path):
    try:
        im = Image.open(path)
        im.load()
        return im
    except Exception as e:
        print(f"ERROR abriendo {path}: {e}")
        return None

def ensure_rgb_with_bg(im, background_color=BACKGROUND_COLOR):
    """
    Convierte P/RGBA a RGB con fondo.
    Usa el snippet que nos diste.
    """
    if im.mode == "P":
        im = im.convert("RGBA")

    if im.mode == "RGBA":
        bg = Image.new("RGB", im.size, background_color)
        bg.paste(im, mask=im.split()[-1])  # usa el alpha como máscara
        im = bg
    else:
        im = im.convert("RGB")
    return im

# Archivos que no terminan en .png 
no_png = [f for f in os.listdir(block_folder) if not f.lower().endswith(".png")]
block_to_drop = [os.path.splitext(i)[0] for i in no_png]

# eliminar en el folder TODOS los archivos cuyo basename esté en block_to_drop
for f in list(os.listdir(block_folder)):
    base = os.path.splitext(f)[0]
    if base in block_to_drop:
        fp = os.path.join(block_folder, f)
        try:
            os.remove(fp)
            print("Eliminado:", fp)
        except Exception as e:
            print("Error eliminando", fp, e)


In [ ]:
def dataset_from_folder(folder_path: str):

    image_files = []
    for f in os.listdir(folder_path):
        image_files.append(f)

    images = []
    labels = []

    for filename in sorted(image_files):
        image_path = os.path.join(folder_path, filename)

        # abrir imagen 
        im = safe_open_image(image_path)
        im = ensure_rgb_with_bg(im)

        buf = io.BytesIO()
        im.save(buf, format="PNG")

        images.append({"bytes": buf.getvalue()})

        # --- generar label desde el nombre ---
        label = os.path.splitext(filename)[0].replace("_", " ")
        labels.append(label)

    # --- dataframe final ---
    df = pd.DataFrame({
        "image": images,
        "label": labels
    })

    return df


In [16]:
df_blocks = dataset_from_folder(folder_path=block_folder)

df_blocks.head()

,image,label
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door bottom
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door top
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia leaves
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia log
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia log top


In [17]:
df_items= dataset_from_folder(folder_path=item_folder)

df_items.head()

,image,label
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia boat
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia chest boat
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia door
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia hanging sign
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,acacia sign


In [18]:
df_all = pd.concat([df_blocks, df_items], ignore_index=True)

In [19]:
df_all.to_parquet("datos/ib_faithful_1.21.1_32.parquet")

### Preproceso para uso con Lora trainer
Guardamos en carpeta con doble archivo por imagen, una con la imagen aumentada a 512x512 y otro con el promt de la imagen

In [20]:
OUTPUT_DIR = "output_dataset"           
OUT_IMAGES = os.path.join(OUTPUT_DIR, "train_images") 
TARGET_RES = (512, 512)                 # necesario para el lora pixel art nerijs
ACTIVATION_WORD = "pixel art"           # activation word del LoRA pixel-art
PROMPT_TEMPLATE = f"{ACTIVATION_WORD} minecraft style of {{label}}"

os.makedirs(OUT_IMAGES, exist_ok=True)

In [21]:
# --- 2) Recolectar rutas de imágenes de item y block ---
def list_images(folder):
    if not os.path.isdir(folder):
        return []
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(".png")]

item_files = list_images(item_folder)
block_files = list_images(block_folder)

print(f"Encontradas {len(item_files)} imágenes en {item_folder}")
print(f"Encontradas {len(block_files)} imágenes en {block_folder}")


Encontradas 604 imágenes en datos/item_1.21.1_32x32
Encontradas 1009 imágenes en datos/block_1.21.1_32x32


In [6]:
# Procesar y guardar 
counter = 0

def save_example(img: Image.Image, label: str, prefix: str, idx: int):
    """
    Guarda imagen y su .txt con prompt en OUT_IMAGES
    """
    base_name = f"{prefix}_{idx:06d}"
    out_img_path = os.path.join(OUT_IMAGES, base_name + ".png")
    out_txt_path = os.path.join(OUT_IMAGES, base_name + ".txt")

    img.save(out_img_path, format="PNG")
    prompt = PROMPT_TEMPLATE.format(label=label)
    with open(out_txt_path, "w", encoding="utf-8") as f:
        f.write(prompt)

# procesar lista de (ruta, label)
for src_list, prefix in [(item_files, "item"), (block_files, "block")]:
    for path in tqdm(src_list, desc=f"Procesando {prefix}"):
        im = safe_open_image(path)
        if im is None:
            continue

        # Normalizar/converter paleta/alpha a fondo blanco
        im = ensure_rgb_with_bg(im, BACKGROUND_COLOR)

        # Determinar label el nombre del archivo (sin extensión)
        filename = os.path.basename(path)
        label_raw = os.path.splitext(filename)[0] 
        label = label_raw.replace("_", " ").strip()

        # Upscale a 512x512 usando NEAREST (mejor para pixel art)
        if im.size != TARGET_RES:
            im = im.resize(TARGET_RES, resample=Image.NEAREST)

        # Guardar
        save_example(im, label, prefix, counter)
        counter += 1

print("Hecho. Total ejemplos guardados:", counter)
print("Directorio listo:", OUT_IMAGES)
print("Formato por imagen: <nombre>.png + <nombre>.txt donde .txt contiene el prompt.")


Procesando block: 100%|██████████| 1009/1009 [00:03<00:00, 266.24it/s]

Hecho. Total ejemplos guardados: 1613
Directorio listo: output_dataset\train_images
Formato por imagen: <nombre>.png + <nombre>.txt donde .txt contiene el prompt.
